<a href="https://colab.research.google.com/github/cheng1103/Test/blob/main/Cnyes_category_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><span style="font-size:20px"><strong>鉅亨網新聞主題爬蟲（精確版，用於Colab平台）</strong></span></h1>

<p>可自訂主題新聞與日期區間爬取新聞內文，並可設定是否爬取頭條以外的新聞<br />
<span style="color:null"><strong>輸出資料的欄位依序為：張貼時間(時間戳)、張貼時間、版別、標題、撰文者、內文、標籤、網址、頭條標記</strong></span><strong>。資料格式為csv檔，並以UTF8編碼，直接存入Google雲端硬碟</strong></p>

<p>支援的分類新聞目錄：</p>

<p>1.台股 V<br />
&nbsp; a.台灣政經 V<br />
&nbsp; b.台股盤勢 V<br />
&nbsp; c.台股盤勢 V<br />
&nbsp; d.台股盤勢 V</p>

<p>2.國際股 V<br />
&nbsp; a.美股 V<br />
&nbsp; b.美股雷達 V<br />
&nbsp; c.歐亞股 V<br />
&nbsp; d.國際政經 V</p>

<p>3.陸港股 V</p>

<p>4.外匯 V</p>

<p>5.期貨 V</p>

<p>6.房產 V</p>

<p>7.基金 V</p>

<p>8.區塊鏈 V</p>


# 新聞類別與查詢區間設定


In [1]:
#設定區域
#==============================================
#頭條設定 
# 1表示限定頭條； 0表示不限 
Headline = 1

#輸入類別
c = '台股'

#查詢區間
s_time = "2022-01-01 00:00:00" #開始日期 
e_time = "2022-01-31 00:00:00" #結束日期

#檔名設定
file_name = 'original_data'

#==============================================

data_URL = []

kind = {
  '台股':'tw_stock',
  '台灣政經':'tw_macro',
  '台股盤勢':'tw_quo',
  '專家觀點':'stock_report',
  '台股新聞':'tw_stock_news',

  '國際股':'wd_stock',
  '美股':'us_stock',
  '美股雷達':'us_stock_live',
  '歐亞股':'eu_asia_stock',
  '國際政經':'wd_macro',

  '陸港股':'cn_stock',
  '外匯':'forex',
  '期貨':'future',
  '房產':'cnyeshouse',
  '基金':'fund',
  '區塊鏈':'bc'

}

c = kind.get(c)


#頭條篩選
if Headline == 1 :
  CH = '&isCategoryHeadline=1'
else :
  CH = ''

# 安裝並載入套件

In [2]:
!pip install fake_useragent
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
from lxml import etree

import random
import time
import datetime
import json
import pandas as pd
#%reset -f

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=689e3076ef35666654db8e72f378946139bd923269b4f5b13b1acd270d8ee0ac
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


# 自定義程式：爬取鉅亨網新聞網址清單

In [19]:
def url_list(s_time,e_time):
  
  #==============================================
  #時間區間設定
  s_time = int(time.mktime(time.strptime(s_time, "%Y-%m-%d %H:%M:%S")))-28800 # 轉成時間戳
  e_time = int(time.mktime(time.strptime(e_time, "%Y-%m-%d %H:%M:%S")))-28800

  #==============================================
  p = 1
  l_page = 9
  data = pd.DataFrame()

  while p <= l_page :

    headers = UserAgent().random #隨機請求表頭
    
    url = 'https://api.cnyes.com/media/api/v1/newslist/category/'+c+'?startAt='+str(s_time)+'&endAt='+str(e_time)+CH+"&limit=30&page="+str(p)
    req = requests.get(url, headers={"User-Agent":headers})
    req = req.json()

    #最末頁之路徑
    if p == 1:
      l_page = req["items"]["last_page"] 

    req_data = req["items"]["data"]
    temp_data = pd.DataFrame.from_dict(req_data)
    
    #擷取資料
    publishAt = temp_data['publishAt']
    newsId = temp_data['newsId']
    category = temp_data['categoryName']
    title = temp_data['title']
    stock = temp_data['stock']
    isCategoryHeadline = temp_data['isCategoryHeadline']
    
    temp_data = pd.concat([publishAt, isCategoryHeadline, category, title, stock, newsId], axis=1)
    temp_data = reFormat(temp_data)
    
    data = pd.concat([data, temp_data], ignore_index=True)
    
    

    #休眠
    t = random.uniform(0.005, 0.1) #隨機數
    time.sleep(t) #暫停

    p = p+1

    if p%20 == 0:
      print(p/l_page*100)
      data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料
      t = random.uniform(0.5, 1) #隨機數
      time.sleep(t)

  data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料
  return(data)



In [21]:
def reFormat (temp_data):
  for r in range(len(temp_data)):

    #擷取股票代號
    temp_data['stock'][r] = ','.join(temp_data['stock'][r])
    if len(temp_data['stock'][r]) == 0 :
      temp_data['stock'][r] = '0'


  return(temp_data)

# 執行程式

In [22]:
data_URL = url_list(s_time,e_time)
data_URL = data_URL.sort_values(by='publishAt', ascending=True)
data_URL = data_URL.reset_index(drop=True) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


42.5531914893617
85.1063829787234


# 檢視新聞網址資料

In [23]:
data_URL

,publishAt,isCategoryHeadline,categoryName,title,stock,newsId
0,1640970025,1,鉅亨新視界,〈元旦新制〉洗發票不給獎、巧克力正名 15項民生新規定一次看,0,4794194
1,1640991635,1,鉅亨新視界,〈元旦新制〉保單借款紓困、理專十誡2.0 八大金融新規上路,0,4794474
2,1641019388,1,台股新聞,世界先進向友達購入L3B廠 完成交割,"5347,2409",4795332
3,1641021326,1,台股新聞,〈圖片集〉台北101跨年倒數有亮點！中國信託銀行動畫字樣迎接2022年,0,4795333
4,1641024044,1,鉅亨新視界,〈觀察〉中國鋼鐵業加速整併 台廠喜憂參半,0,4795145
...,...,...,...,...,...,...
1375,1643616013,1,台股新聞,〈企業衝生育率〉晶技育兒津貼方案 前三胎補助每月3000元起,3042,4807605
1376,1643619640,1,台股新聞,〈企業衝生育率〉鴻海「0到6歲公司養」一胎補助130.5萬元 已發逾3億元,2317,4808165
1377,1643623223,1,台股新聞,〈企業衝生育率〉中華電獨步全台減少員工工時 台灣大津貼最高12萬元,"2412,3045,4904",4802091
1378,1643625030,1,台股新聞,〈企業衝生育率〉多家券商補助生育津貼 最高達10萬元,0,4809088


# 爬取新聞內文

In [24]:
data = pd.DataFrame()
l_page = len(data_URL)

for i in range(l_page):

  headers = UserAgent().random
  url = data_URL['newsId'][i]
  url = 'https://news.cnyes.com/news/id/'+str(url)

  req = requests.get(url, headers={"User-Agent":headers})
  req = req.content.decode()
  req = etree.HTML(req)

  publishAt = data_URL['publishAt'][i]

  d_time = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/div[2]/time/text()')
  d_time = "".join(d_time)

  isCategoryHeadline = data_URL['isCategoryHeadline'][i]

  catalog = data_URL['categoryName'][i]

  title = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/h1/text()')
  title = "".join(title)

  reporter = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/div[2]/span/span/text()')
  reporter = "".join(reporter)

  content = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/div[2]/div/p/text() | //*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/div[2]/div/p/a/text()')
  content = "".join(content)
  if len(content) == 0:
    content = '0'

  stock = data_URL['stock'][i]

  tag = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/nav/a/span/text()')
  tag = ",".join(tag)


  temp_data = {(publishAt , d_time, isCategoryHeadline, catalog, title, reporter, content, stock, tag, url)}
  temp_data = pd.DataFrame(temp_data)
  data = data.append(temp_data)

  t = random.uniform(0.005, 0.1) #隨機數
  time.sleep(t) #暫停

  #列印出進度
  if i % 25 == 0:
    print(i/l_page*100)
    t = random.uniform(0.5, 1) #隨機數
    time.sleep(t)




0.0
55.55555555555556


In [ ]:
  ##############
  headers = UserAgent().random
  url = 'https://news.cnyes.com/news/id/4795333'

  req = requests.get(url, headers={"User-Agent":headers})
  req = req.content.decode()
  req = etree.HTML(req)
  content = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/div[2]/div/p/text() | //*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/div[2]/div/p/a/text()')
  content = "".join(content)
  content

# 檢視新聞內文資料

In [26]:
data.columns = [ 'publishAt', 'time', 'Headline', 'category', 'title', 'reporter', 'content', 'stock', 'tag', 'url']
data

,publishAt,time,Headline,category,title,reporter,content,stock,tag,url
0,1640970025,2022/01/01 01:00,1,鉅亨新視界,〈元旦新制〉洗發票不給獎、巧克力正名 15項民生新規定一次看,鉅亨網新聞中心,邁入 2022 年，國內又有許多新制上路，其中勞權方面影響最多，包括請領勞保年金從 62 歲...,0,"元旦,巧克力,新制,疫苗,基本工資,自販機,勞保,老年年金,洗選蛋,國道,勞工,值班,水稻收入保險",https://news.cnyes.com/news/id/4794194
0,1640991635,2022/01/01 07:00,1,鉅亨新視界,〈元旦新制〉保單借款紓困、理專十誡2.0 八大金融新規上路,鉅亨網新聞中心,2022 年 1 月起，金管會、財政部又有多項金融新制上路，其中，理專十誡 2.0 版本將強...,0,"金管會,保單借款,紓困,元旦,壽險業,遺產稅,當沖,投信",https://news.cnyes.com/news/id/4794474
0,1641019388,2022/01/01 14:43,1,台股新聞,世界先進向友達購入L3B廠 完成交割,鉅亨網記者林薏茹 台北,晶圓代工廠世界先進 (5347-TW) 今 (1) 日宣佈，向友達 (2409-TW) 購入...,"5347,2409","世界先進,半導體,晶圓代工,友達",https://news.cnyes.com/news/id/4795332
0,1641021326,2022/01/01 15:15,1,台股新聞,〈圖片集〉台北101跨年倒數有亮點！中國信託銀行動畫字樣迎接2022年,鉅亨網記者陳蕙綾 台北,0,0,"台北101,跨年,中國信託",https://news.cnyes.com/news/id/4795333
0,1641024044,2022/01/01 16:00,1,鉅亨新視界,〈觀察〉中國鋼鐵業加速整併 台廠喜憂參半,鉅亨網記者彭昱文 台北,中國積極整頓當地鋼鐵產業，不僅去化產能降排外，十四五計畫更推動鋼鐵產業進行整併、打造超大型鋼...,0,"中國,十四五,鋼鐵",https://news.cnyes.com/news/id/4795145
0,1641031220,2022/01/01 18:00,1,鉅亨新視界,〈觀察〉專業分工為王道 為何碳化矽產業是IDM廠的天下？,鉅亨網記者林薏茹 台北,矽基半導體產業已高度垂直分工，台積電 (2330-TW) 更以專業晶圓代工的商業模式，站穩產...,"2330,5483,6488,8086,8255","化合物半導體,半導體,晶圓代工,SiC,基板,IDM",https://news.cnyes.com/news/id/4795133
0,1641053440,2022/01/02 00:10,1,台股新聞,2022開紅盤、CES展登場、大立光/鴻海營收 本周大事預告,鉅亨網記者劉韋廷 台北,告別 2021 年，本周將迎來 2022 年首個交易日，台股開紅盤可望再戰歷史新高，睽違一年...,"2353,3008","台股,本周大事,CES,變種病毒,大立光,鴻海",https://news.cnyes.com/news/id/4795120
0,1641078035,2022/01/02 07:00,1,台股新聞,〈晶技展望〉5G、元宇宙及車用動能強 2022年業績雙位數成長,鉅亨網記者張欽發 台北,晶技 (3042-TW) 2021 年營收、獲利篤定再度改寫新高，晶技看好，2022 年全球...,3042,"晶技,元宇宙,石英元件,車用電子",https://news.cnyes.com/news/id/4795124
0,1641081648,2022/01/02 08:00,1,台股新聞,〈晶技展望〉強攻車用市場 估2022年車用營收比重達12%,鉅亨網記者張欽發 台北,電動車市場崛起，除特斯拉 (TSLA-US) 外，中國及歐美原燃油車廠也積極推出產品，有助汽...,3042,"晶技,石英元件,車用",https://news.cnyes.com/news/id/4795132
0,1641085221,2022/01/02 09:00,1,台股新聞,華碩衝刺AIoT布局 本季不排除再增持瑞傳持股,鉅亨網記者劉韋廷 台北,品牌大廠華碩 (2357-TW) 近年積極布局 AIoT 領域，繼去年斥資 15 億元取得振...,"2357,8114","華碩,AIoT,瑞傳,智慧製造,智慧工廠,品牌,筆電,主板",https://news.cnyes.com/news/id/4795001


# 連結Google Drive並匯出資料

In [27]:
from google.colab import drive
drive.mount('/content/drive')


#輸出檔案
path = '/content/drive/MyDrive/Colab Notebooks/'+file_name+'_full.csv'
data.to_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
